### Import Libraries

In [1]:
#import needed libraries
import warnings
warnings.filterwarnings('ignore')
from   import create_engine
import pyodbc
import pandas as pd
import os
import psycopg2

host= "localhost:5432"
user= "postgres"
password= "zxc"
nombre_db= "DB_Projects"

driver = "{SQL Server}"
server = "DESKTOP-7MLFO65"
database = "AdventureWorks2019"

### Establish Connection

In [2]:
# Source connection: sql server
src_conn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';DATABASE=' + database + ';Trust_Connection=yes')

In [3]:
# Destination: Postgres
url = f"postgresql://{user}:{password}@{host}/{nombre_db}"
engine =create_engine(url)

### Read Data

In [4]:
source = pd.read_sql_query(""" SELECT TOP (10) [BusinessEntityID]
      ,[PersonType]
      ,[NameStyle]
      ,[Title]
      ,[FirstName]
      ,[MiddleName]
      ,[LastName]
      ,[Suffix]
      ,[EmailPromotion]
      ,[AdditionalContactInfo]
      ,[Demographics]
      ,[rowguid]
      ,[ModifiedDate]
  FROM [AdventureWorks2019].[Person].[Person]; """, src_conn)
source

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30
5,6,EM,False,Mr.,Jossef,H,Goldberg,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",0DEA28FD-EFFE-482A-AFD3-B7E8F199D56F,2013-12-16
6,7,EM,False,None,Dylan,A,Miller,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01
7,8,EM,False,None,Diane,L,Margheim,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22
8,9,EM,False,None,Gigi,N,Matthew,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",5FC28C0E-6D36-4252-9846-05CAA0B1F6C5,2009-01-09
9,10,EM,False,None,Michael,None,Raheem,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",CA2C740E-75B2-420C-9D4B-E3CBC6609604,2009-04-26


### Load initial Data to Target | Read Target Data

In [5]:
# Save the data to destination as the intial load. On the first run we load all data.
tbl_name = "stg_IncrementalLoadTest"
source.to_sql(tbl_name, engine, if_exists='replace', index=False)

10

In [6]:
# Read Target data into a dataframe
target = pd.read_sql('Select * from public."stg_IncrementalLoadTest"', engine)
target

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30
5,6,EM,False,Mr.,Jossef,H,Goldberg,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",0DEA28FD-EFFE-482A-AFD3-B7E8F199D56F,2013-12-16
6,7,EM,False,None,Dylan,A,Miller,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01
7,8,EM,False,None,Diane,L,Margheim,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22
8,9,EM,False,None,Gigi,N,Matthew,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",5FC28C0E-6D36-4252-9846-05CAA0B1F6C5,2009-01-09
9,10,EM,False,None,Michael,None,Raheem,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",CA2C740E-75B2-420C-9D4B-E3CBC6609604,2009-04-26


### Read Update Source Data

In [7]:
# Let's select two additional rows from the source. We have two new records
source = pd.read_sql_query(""" SELECT TOP (15) [BusinessEntityID]
      ,[PersonType]
      ,[NameStyle]
      ,[Title]
      ,[FirstName]
      ,[MiddleName]
      ,[LastName]
      ,[Suffix]
      ,[EmailPromotion]
      ,[AdditionalContactInfo]
      ,[Demographics]
      ,[rowguid]
      ,[ModifiedDate]
  FROM [AdventureWorks2019].[Person].[Person]; """, src_conn)
source

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30
5,6,EM,False,Mr.,Jossef,H,Goldberg,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",0DEA28FD-EFFE-482A-AFD3-B7E8F199D56F,2013-12-16
6,7,EM,False,None,Dylan,A,Miller,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01
7,8,EM,False,None,Diane,L,Margheim,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22
8,9,EM,False,None,Gigi,N,Matthew,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",5FC28C0E-6D36-4252-9846-05CAA0B1F6C5,2009-01-09
9,10,EM,False,None,Michael,None,Raheem,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",CA2C740E-75B2-420C-9D4B-E3CBC6609604,2009-04-26


### Update a Source Record. Serve as a modified row

In [9]:
# Also update a record. I will update the middle name Lee
source.loc[source.MiddleName =='Lee', ['MiddleName']] = 'L'
source

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,L,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30
5,6,EM,False,Mr.,Jossef,H,Goldberg,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",0DEA28FD-EFFE-482A-AFD3-B7E8F199D56F,2013-12-16
6,7,EM,False,None,Dylan,A,Miller,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01
7,8,EM,False,None,Diane,L,Margheim,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22
8,9,EM,False,None,Gigi,N,Matthew,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",5FC28C0E-6D36-4252-9846-05CAA0B1F6C5,2009-01-09
9,10,EM,False,None,Michael,None,Raheem,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",CA2C740E-75B2-420C-9D4B-E3CBC6609604,2009-04-26


### Detect Changes in data by comparing source and target

In [10]:
target.apply(tuple,1)

0    (1, EM, False, None, Ken, J, Sánchez, None, 0,...
1    (2, EM, False, None, Terri, Lee, Duffy, None, ...
2    (3, EM, False, None, Roberto, None, Tamburello...
3    (4, EM, False, None, Rob, None, Walters, None,...
4    (5, EM, False, Ms., Gail, A, Erickson, None, 0...
5    (6, EM, False, Mr., Jossef, H, Goldberg, None,...
6    (7, EM, False, None, Dylan, A, Miller, None, 2...
7    (8, EM, False, None, Diane, L, Margheim, None,...
8    (9, EM, False, None, Gigi, N, Matthew, None, 0...
9    (10, EM, False, None, Michael, None, Raheem, N...
dtype: object

In [11]:
source.apply(tuple,1).isin(target.apply(tuple,1))

0      True
1     False
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10    False
11    False
12    False
13    False
14    False
dtype: bool

In [12]:
# detech changes. Get rows that are not present in the target.
changes = source[~source.apply(tuple,1).isin(target.apply(tuple,1))]
changes

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
1,2,EM,False,None,Terri,L,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
10,11,EM,False,None,Ovidiu,V,Cracium,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D2CC2577-EF6B-4408-BD8C-747337FE5645,2010-11-28
11,12,EM,False,None,Thierry,B,D'Hers,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",FA263C7F-600D-4E89-8DCD-0978F3530F5F,2007-12-04
12,13,EM,False,Ms.,Janice,M,Galvin,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",34EB99E0-7042-4DC1-A2FD-BDA290FF0E07,2010-12-16
13,14,EM,False,None,Michael,I,Sullivan,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",9A7501DE-5CAF-4700-AB07-CC81102BB696,2010-12-23
14,15,EM,False,None,Sharon,B,Salavaria,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",BEBA63CB-13F1-4B76-A3DE-FE9AC283A94C,2011-01-11


In [13]:
# Get modified rows
modified = changes[changes.BusinessEntityID.isin(target.BusinessEntityID)]
modified

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
1,2,EM,False,None,Terri,L,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24


In [14]:
# Get new records
inserts = changes[~changes.BusinessEntityID.isin(target.BusinessEntityID)]
inserts

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
10,11,EM,False,None,Ovidiu,V,Cracium,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D2CC2577-EF6B-4408-BD8C-747337FE5645,2010-11-28
11,12,EM,False,None,Thierry,B,D'Hers,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",FA263C7F-600D-4E89-8DCD-0978F3530F5F,2007-12-04
12,13,EM,False,Ms.,Janice,M,Galvin,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",34EB99E0-7042-4DC1-A2FD-BDA290FF0E07,2010-12-16
13,14,EM,False,None,Michael,I,Sullivan,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",9A7501DE-5CAF-4700-AB07-CC81102BB696,2010-12-23
14,15,EM,False,None,Sharon,B,Salavaria,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",BEBA63CB-13F1-4B76-A3DE-FE9AC283A94C,2011-01-11


### Upsert data to target table

In [15]:
def update_to_sql(df, table_name, key_name):
    a = []
    table = table_name
    primary_key = key_name
    temp_table = f"{table_name}_temporary_table"
    for col in df.columns:
        if col == primary_key:
            continue
        a.append(f'"{col}"=s."{col}"')
    df.to_sql(temp_table, engine, if_exists='replace', index=False)
    update_stmt_1 = f'UPDATE public."{table}" f '
    update_stmt_2 = "SET "
    update_stmt_3 = ", ".join(a)
    update_stmt_4 = f' FROM public."{table}" t '
    update_stmt_5 = f' INNER JOIN (SELECT * FROM public."{temp_table}") AS s ON s."{primary_key}"=t."{primary_key}" '
    update_stmt_6 = f' Where f."{primary_key}"=s."{primary_key}" '
    update_stmt_7 = update_stmt_1 + update_stmt_2 + update_stmt_3 + update_stmt_4 + update_stmt_5 +  update_stmt_6 +";"
    print(update_stmt_7)
    with engine.begin() as cnx:
        cnx.execute(update_stmt_7)

In [16]:
# Call update function
update_to_sql(modified, "stg_IncrementalLoadTest", "BusinessEntityID")

UPDATE public."stg_IncrementalLoadTest" f SET "PersonType"=s."PersonType", "NameStyle"=s."NameStyle", "Title"=s."Title", "FirstName"=s."FirstName", "MiddleName"=s."MiddleName", "LastName"=s."LastName", "Suffix"=s."Suffix", "EmailPromotion"=s."EmailPromotion", "AdditionalContactInfo"=s."AdditionalContactInfo", "Demographics"=s."Demographics", "rowguid"=s."rowguid", "ModifiedDate"=s."ModifiedDate" FROM public."stg_IncrementalLoadTest" t  INNER JOIN (SELECT * FROM public."stg_IncrementalLoadTest_temporary_table") AS s ON s."BusinessEntityID"=t."BusinessEntityID"  Where f."BusinessEntityID"=s."BusinessEntityID" ;


In [18]:
target = pd.read_sql('Select * from public."stg_IncrementalLoadTest"', engine)
target

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
2,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
3,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30
4,6,EM,False,Mr.,Jossef,H,Goldberg,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",0DEA28FD-EFFE-482A-AFD3-B7E8F199D56F,2013-12-16
5,7,EM,False,None,Dylan,A,Miller,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01
6,8,EM,False,None,Diane,L,Margheim,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22
7,9,EM,False,None,Gigi,N,Matthew,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",5FC28C0E-6D36-4252-9846-05CAA0B1F6C5,2009-01-09
8,10,EM,False,None,Michael,None,Raheem,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",CA2C740E-75B2-420C-9D4B-E3CBC6609604,2009-04-26
9,2,EM,False,None,Terri,L,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24


In [23]:
def inser_to_sql(df, table_name):
    a = []
    table = table_name
    temp_table = f"{table_name}_temporary_table"
    df.to_sql(temp_table, engine, if_exists='replace', index=False)
    insert_stmt = f'INSERT INTO public."{table}" SELECT * FROM public."{temp_table}";'
    print(insert_stmt)
    with engine.begin() as cnx:
        cnx.execute(insert_stmt)

In [24]:
# Call update function
inser_to_sql(inserts, "stg_IncrementalLoadTest")

INSERT INTO public."stg_IncrementalLoadTest" SELECT * FROM public."stg_IncrementalLoadTest_temporary_table";


In [25]:
target = pd.read_sql('Select * from public."stg_IncrementalLoadTest"', engine)
target

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
2,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
3,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30
4,6,EM,False,Mr.,Jossef,H,Goldberg,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",0DEA28FD-EFFE-482A-AFD3-B7E8F199D56F,2013-12-16
5,7,EM,False,None,Dylan,A,Miller,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",C45E8AB8-01BE-4B76-B215-820C8368181A,2009-02-01
6,8,EM,False,None,Diane,L,Margheim,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",A948E590-4A56-45A9-BC9A-160A1CC9D990,2008-12-22
7,9,EM,False,None,Gigi,N,Matthew,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",5FC28C0E-6D36-4252-9846-05CAA0B1F6C5,2009-01-09
8,10,EM,False,None,Michael,None,Raheem,None,2,None,"<IndividualSurvey xmlns=""http://schemas.micros...",CA2C740E-75B2-420C-9D4B-E3CBC6609604,2009-04-26
9,2,EM,False,None,Terri,L,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
